# DL/ML Training

## Setup

In [ ]:
# !pip install git+https://github.com/sky1ove/katlas.git -Uqq

In [ ]:
# essentials
from fastbook import *
from katlas.imports import *
from katlas.train import *
from katlas.dl import *
import seaborn as sns

# sklearn
from sklearn.neighbors import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.svm import *
from sklearn.ensemble import *
from sklearn.multioutput import MultiOutputRegressor

## Data

In [ ]:
# T5 data
t5_kd = pd.read_parquet('train_data/combine_t5_kd.parquet').reset_index()
t5 = pd.read_parquet('train_data/combine_t5.parquet').reset_index()

# ESM data
esm_kd = pd.read_parquet('train_data/combine_esm_kd.parquet').reset_index()
esm = pd.read_parquet('train_data/combine_esm.parquet').reset_index()

# feature col
t5_col = t5.columns[t5.columns.str.startswith('T5_')]
esm_col = esm.columns[esm.columns.str.startswith('esm_')]

# target col
target_col = t5.columns[~t5.columns.isin(t5_col)][1:] 

In [ ]:
t5_col,esm_col,target_col

(Index(['T5_0', 'T5_1', 'T5_2', 'T5_3', 'T5_4', 'T5_5', 'T5_6', 'T5_7', 'T5_8',
        'T5_9',
        ...
        'T5_1014', 'T5_1015', 'T5_1016', 'T5_1017', 'T5_1018', 'T5_1019',
        'T5_1020', 'T5_1021', 'T5_1022', 'T5_1023'],
       dtype='object', length=1024),
 Index(['esm_0', 'esm_1', 'esm_2', 'esm_3', 'esm_4', 'esm_5', 'esm_6', 'esm_7',
        'esm_8', 'esm_9',
        ...
        'esm_1270', 'esm_1271', 'esm_1272', 'esm_1273', 'esm_1274', 'esm_1275',
        'esm_1276', 'esm_1277', 'esm_1278', 'esm_1279'],
       dtype='object', length=1280),
 Index(['-5P', '-5G', '-5A', '-5C', '-5S', '-5T', '-5V', '-5I', '-5L', '-5M',
        ...
        '4Q', '4N', '4D', '4E', '4s', '4t', '4y', '0s', '0t', '0y'],
       dtype='object', length=210))

## Kfold Split

In [ ]:
source = pd.read_excel('train_data/combine_info_PSPA.xlsx').iloc[:,:2]

info = Data.get_kinase_info().query('pseudo !="1"')

info = source.merge(info,how='left')

In [ ]:
info.source.value_counts()

PSPA    303
KS       87
Name: source, dtype: int64

In [ ]:
# check if training dataset has same order with the info
(info.kinase == t5.kinase).sum(), \
(info.kinase == t5_kd.kinase).sum(), \
(info.kinase == esm.kinase).sum(), \
(info.kinase == esm_kd.kinase).sum()

(390, 390, 390, 390)

In [ ]:
splits = get_splits(info, stratified='group',nfold=5)

StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
# kinase group in train set: 9
# kinase group in test set: 9
---------------------------
# kinase in train set: 312
---------------------------
# kinase in test set: 78
---------------------------
test set: ['EPHA3' 'FES' 'FLT3' 'FYN' 'EPHB1' 'EPHB3' 'FER' 'EPHB4' 'FLT4' 'FGFR1' 'EPHA5' 'TEK' 'DDR2' 'ZAP70' 'LIMK1' 'ULK3' 'JAK1' 'WEE1' 'TESK1' 'MAP2K3' 'AMPKA2' 'ATM' 'CAMK1D' 'CAMK2D' 'CAMK4' 'CAMKK1'
 'CK1D' 'CK1E' 'DYRK2' 'DYRK4' 'HGK' 'IKKE' 'JNK2' 'JNK3' 'KHS1' 'MAPKAPK5' 'MEK2' 'MSK2' 'NDR1' 'NEK6' 'NEK9' 'NIM1' 'NLK' 'OSR1' 'P38A' 'P38B' 'P90RSK' 'PAK1' 'PERK' 'PKCH' 'PKCI' 'PKN1' 'ROCK2'
 'RSK2' 'SIK' 'STLK3' 'TAK1' 'TSSK1' 'ALPHAK3' 'BMPR2' 'CDK10' 'CDK13' 'CDK14' 'CDKL5' 'GCN2' 'GRK4' 'IRE1' 'KHS2' 'MASTL' 'MLK4' 'MNK1' 'MRCKA' 'PRPK' 'QSK' 'SMMLCK' 'SSTK' 'ULK2' 'VRK1']


## Train DL

In [ ]:
seed_everything()

In [ ]:
num_t5 = len(t5_col)
num_esm = len(esm_col)
num_target = len(target_col)

In [ ]:
num_t5,num_esm, num_target

(1024, 1280, 210)

In [ ]:
def mlp_t5():
    return MLP_1(num_t5, num_target)

def mlp_esm():
    return MLP_1(num_esm, num_target)

def cnn_t5():
    return CNN1D_2(num_t5, num_target)

def cnn_esm():
    return CNN1D_2(num_esm, num_target)

In [ ]:
models = {
    'mlp_t5':    (mlp_t5, t5,    t5_col),
    'mlp_t5_kd': (mlp_t5, t5_kd, t5_col),
    'mlp_esm':   (mlp_esm,esm,   esm_col),
    'mlp_esm_kd':(mlp_esm,esm_kd,esm_col),
    'cnn_t5':    (cnn_t5, t5,    t5_col),
    'cnn_t5_kd': (cnn_t5, t5_kd, t5_col),
    'cnn_esm':   (cnn_esm,esm,   esm_col),
    'cnn_esm_kd':(cnn_esm,esm_kd,esm_col)
}

In [ ]:
%%time

oofs={}
metrics_list = []

n_epoch=20
lr = 3e-3

for save_name, (model_func, data, feat_col) in models.items():
    
    oof, metrics = train_dl_cv(data,
                           feat_col,
                           target_col, 
                           splits, 
                           model_func, 
                           save = save_name,
                           n_epoch=n_epoch,lr=lr)
    
    oofs[save_name] = oof
    metrics['model'] = save_name
    metrics_list.append(metrics)

------fold0------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.154047,0.993763,0.080863,0.080229,00:01
1,1.033500,0.957460,0.251623,0.239915,00:01
2,0.908174,0.879915,0.434919,0.403990,00:01
3,0.805384,0.804587,0.560596,0.495304,00:01
4,0.725529,0.720960,0.628532,0.571654,00:01
5,0.656491,0.599209,0.692632,0.614877,00:01
6,0.603124,0.513039,0.724522,0.655832,00:01
7,0.555976,0.479024,0.732275,0.665790,00:01
8,0.513584,0.460103,0.735400,0.682403,00:01
9,0.478237,0.461291,0.735585,0.669326,00:01


overall MSE: 0.4177
Average Pearson: 0.7472 
------fold1------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.140956,0.993235,0.083575,0.075205,00:01
1,1.011993,0.958387,0.244458,0.236904,00:01
2,0.889426,0.886061,0.442374,0.413279,00:01
3,0.795835,0.807503,0.563417,0.515726,00:01
4,0.711169,0.701166,0.652063,0.602737,00:01
5,0.645843,0.616948,0.670980,0.613756,00:01
6,0.590385,0.542628,0.694287,0.611596,00:01
7,0.544477,0.493557,0.716135,0.642213,00:01
8,0.505067,0.495106,0.710593,0.648736,00:01
9,0.469207,0.483776,0.718823,0.653112,00:01


overall MSE: 0.4233
Average Pearson: 0.7386 
------fold2------
lr in training is 0.003


...

overall MSE: 0.3419
Average Pearson: 0.8041 
------fold3------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.222317,0.993610,0.080199,0.032400,00:02
1,1.076854,0.977526,0.150822,0.129489,00:01
2,0.980216,0.998324,0.164397,0.158476,00:02
3,0.913132,0.770909,0.481249,0.440876,00:01
4,0.859254,0.649468,0.593107,0.492686,00:01
5,0.800511,0.686524,0.628884,0.560609,00:01
6,0.739962,0.612573,0.670090,0.540904,00:01
7,0.675889,0.531022,0.705821,0.602978,00:01
8,0.611711,0.439105,0.755904,0.676651,00:01
9,0.552348,0.389951,0.784729,0.703547,00:01


overall MSE: 0.3232
Average Pearson: 0.8073 
------fold4------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.238034,0.995568,0.067444,0.017039,00:02
1,1.087779,0.997393,0.086055,0.001116,00:02
2,0.986789,0.962331,0.221712,0.125929,00:01
3,0.921382,0.781944,0.474640,0.396243,00:02
4,0.856494,0.741954,0.513045,0.422718,00:02
5,0.795871,0.652096,0.633885,0.545384,00:02
6,0.734708,0.623018,0.692585,0.576608,00:02
7,0.667443,0.561336,0.715499,0.607036,00:02
8,0.607193,0.460601,0.746010,0.651354,00:02
9,0.554969,0.463469,0.755858,0.669316,00:01


overall MSE: 0.3379
Average Pearson: 0.8071 
CPU times: user 3min 16s, sys: 17min 23s, total: 20min 40s
Wall time: 22min 42s


In [ ]:
score = pd.concat(metrics_list)

In [ ]:
score.sort_values('mse').head()

,fold,mse,pearson_avg,model
3,3,0.321140,0.814740,cnn_t5_kd
3,3,0.323208,0.807320,cnn_esm_kd
2,2,0.332671,0.812157,cnn_t5_kd
4,4,0.337893,0.807076,cnn_esm_kd
4,4,0.339779,0.801619,cnn_t5_kd


## Train ML

In [ ]:
# Define feature sets and their corresponding column names
feature_sets = {
    't5':(t5,t5_col),
    't5_kd':(t5_kd,t5_col),
    'esm':(esm,esm_col),
    'esm_kd':(esm_kd,esm_col),
}

In [ ]:
# Dictionary of ML models
ml_models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(0.1), # L1 regularization # changing alpha does not change the result
    'Ridge': Ridge(alpha=0.1), # L2 regularization
    'ElasticNet': ElasticNet(0.1), # Combine L1 and L2
    
    # 'SVR':SVR(C=10,gamma=1),
    'DecisionTreeRegressor': DecisionTreeRegressor(), # not very good
    'KNN': KNeighborsRegressor(n_neighbors=3), # compared with others, n=3 gives the best results
    # 'XGBRegressor': XGBRegressor(), #slow without gpu
    # 'RandomForestRegressor': RandomForestRegressor() # slow, use cuml to use gpu
                                      }

In [ ]:
metrics_list2 = []
oofs2 = {}

for model_name, model in ml_models.items():
    for feature_name, (data, feat_col) in feature_sets.items():
        
        # get model name for save
        save_name = f'{model_name}_{feature_name}'
        print('------------------',save_name,'------------------')
        
        
        # if want to save model, just remove save_name
        # oof, metrics = train_ml_cv(data, feat_col, target_col, splits,model,save_name)
        
        # train
        oof, metrics = train_ml_cv(data, feat_col, target_col, splits, model)
        
        # save metrics
        metrics['model'] = save_name
        metrics_list2.append(metrics)
        
        # save oof
        oofs2[save_name] = oof

------------------ LinearRegression_t5 ------------------
------ fold: 0 --------
LinearRegression()
overall MSE: 0.8305
Average Pearson: 0.6931 
------ fold: 1 --------
LinearRegression()
overall MSE: 0.6924
Average Pearson: 0.6997 
------ fold: 2 --------
LinearRegression()
overall MSE: 0.7312
Average Pearson: 0.7086 
------ fold: 3 --------
LinearRegression()
overall MSE: 0.7582
Average Pearson: 0.6908 
------ fold: 4 --------
LinearRegression()
overall MSE: 0.7260
Average Pearson: 0.6874 
------------------ LinearRegression_t5_kd ------------------
------ fold: 0 --------
LinearRegression()
overall MSE: 0.8763
Average Pearson: 0.7168 
------ fold: 1 --------
LinearRegression()
overall MSE: 0.6405
Average Pearson: 0.7313 
------ fold: 2 --------
LinearRegression()
overall MSE: 0.7465
Average Pearson: 0.7429 
------ fold: 3 --------
LinearRegression()
overall MSE: 0.6453
Average Pearson: 0.7328 
------ fold: 4 --------
LinearRegression()
overall MSE: 0.7906
Average Pearson: 0.7263 
-

In [ ]:
score2 = pd.concat(metrics_list2)

## Save results

### metrics

In [ ]:
score2['type'] = 'ML'
score['type']='DL'

In [ ]:
scores = pd.concat([score,score2])

In [ ]:
scores.sort_values('pearson_avg',ascending=False)

,fold,mse,pearson_avg,model,type
3,3,0.321140,0.814740,cnn_t5_kd,DL
2,2,0.332671,0.812157,cnn_t5_kd,DL
3,3,0.323208,0.807320,cnn_esm_kd,DL
4,4,0.337893,0.807076,cnn_esm_kd,DL
2,2,0.341916,0.804054,cnn_esm_kd,DL
...,...,...,...,...,...
2,2,0.870463,0.361619,Lasso_t5,ML
4,4,0.871231,0.359918,ElasticNet_t5,ML
4,4,0.871660,0.359292,ElasticNet_t5_kd,ML
4,4,0.871864,0.359087,Lasso_t5,ML


In [ ]:
scores.to_csv('supp/scores.csv',index=False)

### oof

In [ ]:
len(oofs2),len(oofs)

(24, 8)

In [ ]:
oofs2.keys(),oofs.keys()

(dict_keys(['LinearRegression_t5', 'LinearRegression_t5_kd', 'LinearRegression_esm', 'LinearRegression_esm_kd', 'Lasso_t5', 'Lasso_t5_kd', 'Lasso_esm', 'Lasso_esm_kd', 'Ridge_t5', 'Ridge_t5_kd', 'Ridge_esm', 'Ridge_esm_kd', 'ElasticNet_t5', 'ElasticNet_t5_kd', 'ElasticNet_esm', 'ElasticNet_esm_kd', 'DecisionTreeRegressor_t5', 'DecisionTreeRegressor_t5_kd', 'DecisionTreeRegressor_esm', 'DecisionTreeRegressor_esm_kd', 'KNN_t5', 'KNN_t5_kd', 'KNN_esm', 'KNN_esm_kd']),
 dict_keys(['mlp_t5', 'mlp_t5_kd', 'mlp_esm', 'mlp_esm_kd', 'cnn_t5', 'cnn_t5_kd', 'cnn_esm', 'cnn_esm_kd']))

In [ ]:
OOF = {**oofs, **oofs2}

In [ ]:
len(OOF)

32

In [ ]:
# to save the dictionary
save_pickle('raw/oof.pkl',OOF)

In [ ]:
# to load
dd  = load_pickle('raw/oof.pkl')

len(dd)

32